# Transportligninger

I innføringen kalte vi ligningen

$$
u_x = u_y
$$

for en transportligning. La oss tenke oss at en av variablene er tid. En mer generell versjon av samme ligning er

$$
u_t + a u_x = 0
$$

for et tall $a$. Dette er ligningen som vi skal studere nå. Senere vil vi se på den enda mer generelle ligningen

$$
u_t + a(x,t,u)u_x = g(x,t),
$$

og den nært beslektede *bevaringsloven*

$$
u_t + f(x,t,u)_x = g(x,t)
$$

## 1. En analytisk løsning

### Oppgave 

Sjekk med derivasjon at $u(x,t) = g(x-at)$ løser ligningen 
$$
u_t + a u_x = 0
$$
for hvilken som helst envariabelfunksjon $g$.

### a) Med initalbetingelser

Anta at vi har en initialbetingelse av formen $u(x,0) = f(x)$.

Sammenlign det med løsningen $u(x,t) = g(x-at)$; vi setter inn $t=0$ og får $u(x,0)=g(x)$. Dvs. vi må sette $f=g$ for at initialbetingelsen skal være tilfredsstilt. Løsningen er altså

$$
u(x,t) = f(x-at)
$$

Hva betyr det? Vi ser at løsningen flytter på seg, med hastighet (inkludert retning) bestemt av $a$. Prøv koden under. Prøv også å endre på
funksjonen 'f(x)' og hastigheten 'a'.

In [12]:
import numpy as np
import matplotlib.pyplot as plt

# dette er funksjonen vi skal bruke
# du kan erstatte med en annen funksjon om du vil
def f(x):
    return np.sin(x)
     
# vi lager verdiene av x og t
x = np.arange(0, 2*np.pi, 0.01)
t = np.arange(0, 2*np.pi, 0.05)

# lager rutenettet
X, T = np.meshgrid(x,t)

# regner ut funksjonen på rutenettet
a=1   # hastighet
u = f(X-a*T)

In [ ]:
## animasjon

import matplotlib.animation as animation
from IPython.display import HTML

fig, ax = plt.subplots()

# initialplott, for tid t=0
line, = ax.plot(x, u[0,:])

# oppdater plott, for andre tider t_i
def animate(i):
    line.set_ydata(u[i,:])
    return line,

# animasjon
# interval bestemmer hvor fort det vises
# save_count hvor mange tider t_i som inngår
ani = animation.FuncAnimation(fig, animate, interval=20, blit=True, save_count=t.size)


HTML(ani.to_jshtml())

Det er også nyttig å se på nivåkurveplot av funksjonen $u(x,t)$ (også kalt kontourplott). Nivåkurvene har et spesielt navn i denne sammenhengen: **karakteristikker**. 

Siden vi kjenner verdien av løsningen fra initialbetingelsene, så vet vi hvor nivåkurvene skjærer $x$-aksen. Da kan vi finne løsningen for alle tidspunkt ved å gå langs karakteristikkene.

In [ ]:
# kontourplott

plt.contour(X,T,u)
plt.xlabel("x")
plt.ylabel("t")
plt.colorbar()

### b) Med randbetingelser

Et typisk randverdiproblem er:

$$
u_t + a u_x = 0, \quad 0<x<1, \quad u(x,0)=f(x), \quad  u(0,t)=h(t)
$$

Løsningen er fortsatt $u(x,t) = g(x-at)$, men ved å sette $x=0$ får vi $u(0,t) = g(-at)$, slik at $g(-at) = h(t)$. Dermed ser vi at $g(x-at) = g(-at)$ slik at $t$ må tilfredsstille $t = -\frac{x-at}{a}$ og vi får

**Alternativ 1**

$$
u(x,t) = h\left( -\frac{x-at}{a}\right).
$$

Men hva med den gamle løsningen

**Alternativ 2**
$$
u(x,t) = f(x-at)?
$$

Vi kan forstå svaret bedre med å se på konturene. Det viser seg at løsningen $u(x,t)$ er en av de to alternativene over, avhengig av verdien til $x, t$. 


### Oppgave

Prøv å forstå fra plotten under at alternativ 1 inntreffer om $x < at$, mens om $x > at$ får vi alternativ 2.

In [ ]:
## nytt rutenett
x = np.arange(0, 1, 0.01)
t = np.arange(0, 1, 0.01)
X, T = np.meshgrid(x,t)

## kontourplott
def h(x):
    return 2+np.sin(x)

def u(x,t):
    return np.where(x > a*t, f(x-a*t), h(-(x-a*t)/a))

v = u(X,T)

# Hvilken verdier vi ønsker å se på nivåkurvene til
levels = np.linspace(-1,3,40)

plt.contour(X,T,v, levels=levels)
plt.xlabel("x")
plt.ylabel("t")
plt.colorbar()

## 2. Lax-Friedrichs metode

Vår 'fasit' på hva løsningen på $u$ for denne partielle differensialligningen blir metoden:

$$
u^{i+1}_j = \frac{1}{2}(u^i_{j+1} + u^i_{j-1}) + \frac{ak}{2h} u^i_{j-1} - \frac{ak}{2h} u^i_{j+1}
$$

som heter **Lax-Friedrichs metode**. Det er ikke nødvendigvis den mest nøyaktige metoden, men den er pålitelig, lite krevende å implementere, og kan lett tilpasses til mer generelle bevaringslover (se senere).

Formelen leses slik: $u^i_j$ er verdien til funksjonen ved tidssteg nummer $i$ og $x$-posisjon nummer $j$, dvs. $u_j^i = u(x_j,t_i)$. Når vi begynner så kjenner vi alle funksjonsverdiene ved starttiden, dvs. $u^0_j$ for alle $j$. Formelen forteller oss derfor hvordan vi finner alle funksjonsverdiene ett tidsteg frem i tid. 

I tillegg kommer randbetingelsene, som ikke er tatt med i formelen. 

In [16]:
## Funksjon som gjennomfører et tidssteg med Lax-Friedrichs metode

# a hastighet fra transportligningen
# fa randbetingelser på venstre side, en funksjon av tid
# fb randbetingelser på høyre side, en funksjon av tid
# h steglengde i rom
# k steglengde i tid
# t nåværende tid
# ui funksjonsverdiene for nårværende tidspunkt.
# unext funksjonsverdiene for neste tidspunkt, dvs. ui+1.
def laxfrid(a, fa, fb, ui, h, k, t):
    n = ui.size
    unext = np.zeros(n)
    for j in np.arange(1,n-1):
        unext[j] = (ui[j+1]+ui[j-1])/2 + (a*k/(2*h))*ui[j-1] - (a*k/(2*h))*ui[j+1]
    unext[0] = fa(t)
    unext[-1] = fb(t)
    return unext

In [17]:
# randbetingelser venstre side
def fa(t):
    return np.sin(np.pi*t)

# randbetingelser høyre side
def fb(t):
    return np.sin(np.pi*t)

# setter hastighet a
hastighet = 1

# setter opp rutenettet med punkter i tid og rom
# n=200 punkter mellom 0 og T=1 i tid
# m=50 punkter mellom a=1 og b=-1 i rom
T = 1
n = 200

a = -1
b = 1
m = 50

# verdiene for t og x
t = np.linspace(0,T,n)
x = np.linspace(a,b,m)

# beregn k og h
k = t[1]-t[0]
h = x[1]-x[0]

# initialiserer u
u = np.zeros((n,m))

# initialbetingelser
u[0,:] = np.sin(np.pi*x)

# vi kjører lax-friedrichs metode
for i in np.arange(1,n):
    u[i,:] = laxfrid(hastighet, fa, fb, u[i-1,:],h,k,t[i])    

In [ ]:
# Vi plotter initialbetingelsen og en animasjon av løsningen vi regnet ut over
fig, ax = plt.subplots()
line, = ax.plot(x, u[0,:])

def animate(i):
    line.set_ydata(u[i,:])
    return line,

ani = animation.FuncAnimation(fig, animate, interval=20, blit=True, save_count=t.size)

from IPython.display import HTML
HTML(ani.to_jshtml())